In [2]:
import yfinance as yf
import sqlite3
from sqlite3 import Error

In [60]:
DB_PATH = "chris_yfinance_testing.db"

class DBCursor:
    
    """
    Matt's implementation of cursor context manager 
    """
    
    def __init__(self):
        self.db_path = DB_PATH

    def __enter__(self):
        self.connection = sqlite3.connect(self.db_path)
        self.cursor = self.connection.cursor()
        return self.cursor

    def __exit__(self, exc_type, exc_value, traceback):
        self.connection.commit()
        self.connection.close()
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, traceback)
        return


class SecuritiesDB:

    def __init__(self):
        self.db_file = DB_PATH

    def initialize_schema(self):

        security_table = """CREATE TABLE IF NOT EXISTS security (
            ticker TEXT PRIMARY KEY,
            name TEXT,
            exchange TEXT,
            currency TEXT,
            type TEXT
            ) 
            """

        exchange_table = """CREATE TABLE IF NOT EXISTS exchange (
            ticker TEXT,
            exchange_name TEXT,
            exchange_timezone TEXT,
            exchange_timezone_short TEXT
            )
            """

        price_per_day = """CREATE TABLE IF NOT EXISTS price_daily (
            date TEXT PRIMARY KEY,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            adjusted_close REAL,
            volume INTEGER,
            security_id TEXT,
            FOREIGN KEY(security_id) REFERENCES security (ticker)
            )
            """
        
        price_per_minute = """CREATE TABLE IF NOT EXISTS price_minutely (
            date TEXT PRIMARY KEY,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            adjusted_close REAL,
            volume INTEGER,
            security_id TEXT,
            FOREIGN KEY(security_id) REFERENCES security (ticker)
            )
            """

        company_table = """CREATE TABLE IF NOT EXISTS company (
            name TEXT PRIMARY KEY,
            sector TEXT,
            hq_country TEXT,
            security_id TEXT,
            FOREIGN KEY (security_id) REFERENCES security (ticker)
            )
            """

        # make empty tables
        tables = [security_table, exchange_table, price_per_day, price_per_minute, company_table]
        for table in tables:
            self.__create_table(table)

    def add_ticker(self, symbol):
        
        yf_ticker = yf.Ticker(symbol)
        with DBCursor() as cursor:
            
            # populate security table
            security_attributes = (symbol,
                                    yf_ticker.info['longName'],
                                    yf_ticker.info['exchange'],
                                    yf_ticker.info['currency'],
                                    yf_ticker.info['quoteType']) 

            cursor.execute("INSERT INTO security VALUES (?,?,?,?,?)", security_attributes)

            # populate exchange table
            exchange_attributes = (symbol,
                                    yf_ticker.info['exchange'],
                                    yf_ticker.info['exchangeTimezoneName'],
                                    yf_ticker.info['exchangeTimezoneShortName'])

            cursor.execute("INSERT INTO exchange VALUES (?,?,?,?)", exchange_attributes)

            # populate company table
            name = yf_ticker.info['longName']
            sector = yf_ticker.info['sector']
            hq_country = yf_ticker.info['country']
            security_id = symbol
            company_attributes = (name, sector, hq_country, security_id)

            cursor.execute("INSERT INTO company VALUES (?,?,?,?)", company_attributes)

            # populate price_daily table
            
            time_series_daily = yf.download(symbol, period='1d', interval='1d', threads='true')
            security_id = ticker
            time_series_daily['security_id'] = [security_id] * len(time_series_daily.index)
            time_series_daily.index = time_series_daily.index.strftime("%Y-%m-%d %H:%M:%S")

            time_series_formatted = time_series_daily.itertuples()
            data = tuple(time_series_formatted)

            wildcards = ','.join(['?'] * 8)

            cursor.executemany("INSERT INTO price_daily VALUES (%s)" % wildcards, data)
                
            # populate price_minutely table
            
            time_series_minutely = yf.download(symbol, period='1d', interval='1m', threads='true')
            security_id = ticker
            time_series_minutely['security_id'] = [security_id] * len(time_series_minutely.index)
            time_series_minutely.index = time_series_minutely.index.strftime("%Y-%m-%d %H:%M:%S")

            time_series_formatted = time_series_minutely.itertuples()
            data = tuple(time_series_formatted)

            wildcards = ','.join(['?'] * 8)

            cursor.executemany("INSERT INTO price_minutely VALUES (%s)" % wildcards, data)
            
    def get_table(self, tablename):
        
        with DBCursor() as cursor:
            cursor.execute("SELECT rowid,* FROM %s"%tablename)
            fulltable = cursor.fetchall()

        return fulltable
        

    def __create_table(self, create_table_sql):

        with DBCursor() as cursor:
            
            try:
                cursor.execute(create_table_sql)
            except Error as e:
                print(e)


In [61]:
stocksDB = SecuritiesDB()
stocksDB.initialize_schema()

In [62]:
ticker="MSFT"

stocksDB.add_ticker(ticker)
print("finished")
#EquitiesDB.populate_schema_by_tickers(stocksDB, tickers)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
finished


In [66]:
stocksDB.get_table("security")
stocksDB.get_table("exchange")
stocksDB.get_table("company")
stocksDB.get_table("price_daily")
stocksDB.get_table("price_minutely")[0:3]


[(1,
  '2021-11-24 09:30:00',
  335.5400085449219,
  335.5400085449219,
  335.5400085449219,
  335.5400085449219,
  335.5400085449219,
  949761,
  'MSFT'),
 (2,
  '2021-11-24 09:31:00',
  335.510009765625,
  335.6600036621094,
  334.9659118652344,
  335.2200012207031,
  335.2200012207031,
  106305,
  'MSFT'),
 (3,
  '2021-11-24 09:32:00',
  335.1700134277344,
  335.1700134277344,
  334.614501953125,
  334.8699951171875,
  334.8699951171875,
  65182,
  'MSFT')]